In [1]:
from pynq import Overlay

overlay = Overlay('demapping.bit')

In [2]:
overlay?

In [3]:
top_func = overlay.demapping_0

In [4]:
top_func.register_map

RegisterMap {
  A = Register(A=write-only),
  B = Register(B=write-only),
  I = Register(I=4294967293),
  I_ctrl = Register(I_ap_vld=1, RESERVED=0),
  Q = Register(Q=4294967293),
  Q_ctrl = Register(Q_ap_vld=1, RESERVED=0)
}

In [10]:
%%time
def demapping(A, B):
    # Define the mapping table
    mapping_table = [
        [-3, -3], [-3, -1], [-3, 3], [-3, 1],
        [-1, -3], [-1, -1], [-1, 3], [-1, 1],
        [3, -3], [3, -1], [3, 3], [3, 1],
        [1, -3], [1, -1], [1, 3], [1, 1]
    ]
    
    min_dist = 10000.0  # Arbitrary large value for initialization
    min_index = 0
    
    # Loop to calculate the closest mapping point
    for i in range(16):
        dist = ((A - mapping_table[i][0] * 100) ** 2 +
                (B - mapping_table[i][1] * 100) ** 2)
        
        if min_dist > dist:
            min_dist = dist
            min_index = i
    
    # Get the I and Q values
    I = mapping_table[min_index][0]
    Q = mapping_table[min_index][1]
    
    return I, Q

# Example usage
A = 150
B = -250
I, Q = demapping(A, B)
print(f"I: {I}, Q: {Q}")


I: 1, Q: -3
CPU times: user 2.5 ms, sys: 0 ns, total: 2.5 ms
Wall time: 3.52 ms


In [17]:
%%time
top_func.register_map.A = 150
top_func.register_map.B = 150


val = top_func.read(0x30)
val

CPU times: user 413 µs, sys: 51 µs, total: 464 µs
Wall time: 514 µs


1